Assignment 4 \
Part 3: DT-Wage Regression \
Submitted by Group 5 \

Groupmembers:

1. Sajan Kumer Sarker (2111131642) (sajan.sarker@northsouth.edu)
2. Rafsan Jani Chowdhury (2011424642) (rafsan.chowdhury@northsouth.edu)
3. Rosely Mohammad (2014219642) (rosely.mohammad@northsouth.edu)

Task: \
1) Experiment with Decision Trees and XGBoost

2) Understanding of Evaluation metrics



**For Regression problem: Mean-squared error**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor

In [3]:
df = pd.read_csv('/content/drive/MyDrive/CSE445-Assignment/Assignment-4/dataset/DT-Wage.csv')
print(df.shape)
print(df.head())

(3000, 11)
   year  age            maritl      race        education              region  \
0  2006   18  1. Never Married  1. White     1. < HS Grad  2. Middle Atlantic   
1  2004   24  1. Never Married  1. White  4. College Grad  2. Middle Atlantic   
2  2003   45        2. Married  1. White  3. Some College  2. Middle Atlantic   
3  2003   43        2. Married  3. Asian  4. College Grad  2. Middle Atlantic   
4  2005   50       4. Divorced  1. White       2. HS Grad  2. Middle Atlantic   

         jobclass          health health_ins   logwage        wage  
0   1. Industrial       1. <=Good      2. No  4.318063   75.043154  
1  2. Information  2. >=Very Good      2. No  4.255273   70.476020  
2   1. Industrial       1. <=Good     1. Yes  4.875061  130.982177  
3  2. Information  2. >=Very Good     1. Yes  5.041393  154.685293  
4  2. Information       1. <=Good     1. Yes  4.318063   75.043154  


In [4]:
print('Number of missing values: ')
print(df.isnull().sum())

Number of missing values: 
year          0
age           0
maritl        0
race          0
education     0
region        0
jobclass      0
health        0
health_ins    0
logwage       0
wage          0
dtype: int64


In [5]:
df = pd.get_dummies(df, columns=['maritl', 'race', 'education', 'region', 'jobclass', 'health', 'health_ins'])
df.columns = df.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')
print(df.shape)
print(df.head())

(3000, 25)
   year  age   logwage        wage  maritl_1. Never Married  \
0  2006   18  4.318063   75.043154                     True   
1  2004   24  4.255273   70.476020                     True   
2  2003   45  4.875061  130.982177                    False   
3  2003   43  5.041393  154.685293                    False   
4  2005   50  4.318063   75.043154                    False   

   maritl_2. Married  maritl_3. Widowed  maritl_4. Divorced  \
0              False              False               False   
1              False              False               False   
2               True              False               False   
3               True              False               False   
4              False              False                True   

   maritl_5. Separated  race_1. White  ...  education_3. Some College  \
0                False           True  ...                      False   
1                False           True  ...                      False   
2           

In [6]:
X = df.drop(columns=['wage'])
y = df['wage']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f'Train Data Shape (X, y): {X_train.shape, y_train.shape}')
print(f'Validation Data Shape (X, y): {X_val.shape, y_val.shape}')
print(f'Test Data Shape (X, y): {X_test.shape, y_test.shape}')

Train Data Shape (X, y): ((2100, 24), (2100,))
Validation Data Shape (X, y): ((450, 24), (450,))
Test Data Shape (X, y): ((450, 24), (450,))


In [16]:
# initialize the regressor models
dt_regressor = DecisionTreeRegressor(random_state=42)
xgb_regressor = XGBRegressor(random_state=42)

# initialize hyperparameters for gridsearchCV
dt_param_grid = {
    'max_depth': [2,3,4,5,8,10],
    'min_samples_split': [2,4,5,10],
    'min_samples_leaf': [1,3,5,8,10,12]
}

xgb_param_grid = {
    'max_depth': [1,3,5,7,10],
    'learning_rate': [0.1,0.01,0.001],
    'n_estimators': [10000,15000,16000],
    'subsample': [0.8,1.0]
}

In [8]:
# Decision Tree:
dt_grid_search = GridSearchCV(estimator=dt_regressor, param_grid=dt_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
dt_grid_search.fit(X_train, y_train)

dt_best_params = dt_grid_search.best_params_
print(f"Decision Tree Best Parameters: {dt_best_params}")

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=2, min_samples

In [9]:
# XGBoost:
xgb_grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=xgb_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
xgb_grid_search.fit(X_train, y_train)

xgb_best_params = xgb_grid_search.best_params_
print(f"XGBoost Best Parameters: {xgb_best_params}")

Fitting 3 folds for each of 90 candidates, totalling 270 fits
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   3.4s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   8.6s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   7.0s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=  11.6s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=   3.7s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=   3.7s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   8.4s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   3.6s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   3.5s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=1.0; total t

In [10]:
# now retrain the model on the combined training and validation set with the best hyperparameters
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

dt_final_model = DecisionTreeRegressor(**dt_best_params, random_state=42)
xgb_final_model = XGBRegressor(**xgb_best_params, random_state=42)

In [11]:
# retrain decision tree final model
dt_final_model.fit(X_train_val, y_train_val)

DecisionTreeRegressor(max_depth=10, random_state=42)

In [12]:
# retrain xgboost final model
xgb_final_model.fit(X_train_val, y_train_val)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=1, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=16000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
# evaluate the final model on test set
y_test_pred_dt = dt_final_model.predict(X_test)
y_test_pred_xgb = xgb_final_model.predict(X_test)

In [15]:
def mse(y_actual, y_pred):
    error = 0.0

    for actual, predicted in zip(y_actual, y_pred):
        error += (actual - predicted) ** 2
    mse = error / len(y_actual)
    return mse

print(mse(y_test, y_test_pred_dt))
print(mse(y_test, y_test_pred_xgb))

0.6653882185096041
2.602776978987605
